In [1]:
from tl_tools import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
from tensorflow.keras.applications import NASNetMobile

In [2]:
setup_mixed_precision()
train_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/data/USTC_SmokeRS/processed/train'

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True ,vertical_flip=True)
train =train_datagen.flow_from_directory(train_dir, color_mode= 'rgb', batch_size = 16, shuffle= True, seed = 1, target_size=(256, 256))


X_train, y_train, X_val, y_val = train_val_split(train, val_split= 0.3)

Mixed precision policy set to: mixed_float16

Found 4980 images belonging to 6 classes.
Number of batches in the training data: 312
Batch size of a single batch 16
Number of samples in the training dataset 4980

Number of training data batches with val split of 0.3: 219
Number of validation data batches: 93



2024-11-11 17:55:49.392257: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-11-11 17:55:49.392285: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-11-11 17:55:49.392295: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-11-11 17:55:49.392311: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-11 17:55:49.392324: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Shape of image training set: (3504, 256, 256, 3)
Shape of image validation set: (1476, 256, 256, 3)

Shape of label training set: (3504, 6)
Shape of label validation set: (1476, 6)


In [3]:
test_dir = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/data/USTC_SmokeRS/processed/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test = test_datagen.flow_from_directory(test_dir, color_mode= 'rgb', batch_size = 16, shuffle= True, seed = 1, target_size=(256, 256))
X_test, y_test = test_splits(test)

Found 1245 images belonging to 6 classes.
Number of batches in the test data: 78
Batch size of a single batch 16
Number of samples in the test dataset 1245

Shape of image test set: (1245, 256, 256, 3)

Shape of label test set: (1245, 6)


In [4]:
pooling = 'avg'
nasnetmobile = NASNetMobile(weights= 'imagenet',include_top= False, input_shape= (256, 256, 3), pooling = pooling)
nasnetmobile.trainable = False

model = fc_layers(nasnetmobile, name= 'nasnet_mobile')

In [5]:
model.compile(optimizer=Adam(learning_rate = 0.001, clipnorm= 1.0),
                     loss='categorical_crossentropy', 
                     metrics=['accuracy', 'F1Score'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=True, start_from_epoch=20)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-6)

history =  model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, verbose=1, 
                            callbacks= [early_stopping, reduce_lr])

Epoch 1/50


2024-11-11 17:56:00.560019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


110/110 ━━━━━━━━━━━━━━━━━━━━ 60s 394ms/step - F1Score: 0.4274 - accuracy: 0.4396 - loss: 1.5810 - val_F1Score: 0.7180 - val_accuracy: 0.7215 - val_loss: 0.8534 - learning_rate: 0.0010
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 223ms/step - F1Score: 0.6865 - accuracy: 0.6961 - loss: 0.8517 - val_F1Score: 0.7653 - val_accuracy: 0.7663 - val_loss: 0.6829 - learning_rate: 0.0010
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 226ms/step - F1Score: 0.7280 - accuracy: 0.7324 - loss: 0.7286 - val_F1Score: 0.7868 - val_accuracy: 0.7893 - val_loss: 0.6055 - learning_rate: 0.0010
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 227ms/step - F1Score: 0.7588 - accuracy: 0.7621 - loss: 0.6836 - val_F1Score: 0.7867 - val_accuracy: 0.7940 - val_loss: 0.6189 - learning_rate: 0.0010
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 25s 230ms/step - F1Score: 0.8082 - accuracy: 0.8126 - loss: 0.5302 - val_F1Score: 0.7908 - val_accuracy: 0.7947 - val_loss: 0.5717 - learning_rate: 0.0010
Epoch 6/50
110/110 ━━━━━━━━━━━━━━━━━

In [6]:
path = '/Users/pimpijnenburg/Desktop/Thesis/USTC_SmokeRS_dataset/code/transfer_learning/trained_models'
model_save_path = os.path.join(path, 'nasnet_mobile.keras')
model.save(model_save_path) 